<a href="https://colab.research.google.com/github/d-m-bailey/ihp-mpw-be/blob/bandgap_ref_cmos/TO_202504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend Verification for the iHP April 2025 Open MPW Shuttle

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/) [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-connection checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# iHP Open-MPW Data
The 17 submissions to iHp open mpw shuttle for April 2025 listed below are located [here](https://github.com/IHP-GmbH/TO_Apr2025).

* 160GHz_LNA
* 40_GHZ_LOW_NOISE_TIA
* 6502-cpu
* 97_GHZ_LINEAR_TIA
* DC_to_130_GHz_TIA
* GPS_LNA
* Greyhound
* Mixer5GHz
* PA_180GHz
* TTIHP0p2
* TTIHP25a
* VCO_130nm_LSI
* active_L_VCOs
* ascon
* bandgap_ref_cmos
* elemrv-n
* i2c-gpio-expander



In [17]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export LVS_ROOT=/content/extra_be_checks

export MPW=TO_Apr2025
export PROJECT=
export GDSFILE=
export TOP=
unset WORK_ROOT

Overwriting /content/env


# User setup
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

(Takes about 3 minutes.)

In [12]:
%%shell
cd
cat /content/env
source /content/env

lsb_release -a 2>/dev/null

if [[ ! -d $PDK_ROOT ]]; then
  echo "==> Installing ciel..."
  pip install ciel
fi
ciel enable --pdk $PDK $PDK_COMMIT
# patch for mimcap and antenna error in netgen setup file from pdk
sed -i '/lsearch .cells2/,/circuit2/s/circuit1 .dev/circuit2 $dev/' $PDK_ROOT/$PDK/libs.tech/netgen/ihp-sg13g2_setup.tcl
echo "==> Using pdk $PDK commit $PDK_COMMIT (patched)
"

if [[ ! -d extra_be_checks ]]; then
  echo "==> Downloading extra_be_checks $EXTRA_CHECK_COMMIT"
  rm -rf extra_be_checks
  git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_COMMIT
fi
echo "==> Using extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)
"

if ! command -v netgen; then
  echo "==> Downloading and installing netgen $NETGEN_COMMIT"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_COMMIT
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using netgen version $(netgen -batch | awk '/Netgen/ {print $2}')
"

if ! command -v magic; then
  echo "==> Downloading and installing magic $MAGIC_COMMIT"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_COMMIT
  cd magic
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using magic version $(magic -dnull -noc --version)
"

if ! command -v klayout; then
  echo "==> Downloading and installing klayout $KLAYOUT_DEB for $KLAYOUT_DISTRO"
  wget -P /root https://www.klayout.org/downloads/$KLAYOUT_DISTRO/$KLAYOUT_DEB
  sudo apt install /root/$KLAYOUT_DEB
  cd
fi
echo "==> Using $(klayout -v)
"

if ! command -v cvc_rv; then
  echo "==> Downloading and installing cvc_rv $CVC_COMMIT"
  sudo apt install autopoint bison flex
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_COMMIT
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
echo "==> Using $(cvc_rv -v)
"

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
echo $PATH
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
#export PRECHECK_ROOT=/content/mpw_precheck
#export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=TO_Apr2025
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/local/bin
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
⠋ Enabling version ddb601a4a4473163e1ed6df4

Create a script to extract the user_project_wrapper hierarchy as gds from the caravel oasis.

Only needs to be run once.

In [ ]:
%%writefile oas2gds.py
import pya

app = pya.Application.instance()
opt = pya.SaveLayoutOptions()
layout_view = pya.Layout()

input_layout = "caravel.oas"
# Setting the name of the output file and setting the substitution character
print("[INFO] Reading from " + input_layout)

# Reading the input file and writing it to the output file name
layout_view.read(input_layout)
for cell_it in layout_view.each_cell():
    if cell_it.name.endswith("project_wrapper"):
        myIndex = layout_view.cell(cell_it.name).cell_index()
        output = "work/gds/" + cell_it.name + ".gds.gz"
        opt.set_format_from_filename(output)
        opt.oasis_substitution_char=''
        break
opt.select_cell(myIndex)
print("[INFO] Writing " + output)
layout_view.write(output, opt)

app.exit(0)

Writing oas2gds.py


# Clone the design repo and list the gds files.

GDS files are also saved in /content/gdsfiles.txt.

In [16]:
%%shell
cat /content/env
source /content/env
if [[ ! -d data ]]; then
  git clone https://github.com/IHP-GmbH/TO_Apr2025.git data
fi
cd data
tee /content/gdsfiles.txt <<EOF
   Size Project                   GDS file
------- ------------------------- ---------------------------------------------
EOF
ls -s $(find . -type d -name gds -exec find {} -type f \;) |
  sed -e 's,\./,,' -e 's,/, ,' |
  awk '{printf "%7d %-25s %s\n", $1, $2, $3}' |
  tee -a /content/gdsfiles.txt

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
echo $PATH
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
#export PRECHECK_ROOT=/content/mpw_precheck
#export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=TO_Apr2025
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/local/bin
   Size Project                   GDS file
------- ------------------------- ---------------------------------------------
   6756 160GHz_LNA                de

# List verilog and spice files

In [ ]:
%%shell
source /content/env
cd data
find . -name '*.spice'
find . -name '*.v'

./spi/lvs/user_project_wrapper.spice
./spi/lvs/user_proj_example.spice
./mpw_precheck/outputs/user_project_wrapper.filtered.v
./verilog/rtl/uprj_netlists.v
./verilog/rtl/main.v
./verilog/rtl/user_project_wrapper.v
./verilog/rtl/L1_cache.v
./verilog/rtl/user_proj_example.v
./verilog/rtl/memory_trace.v
./verilog/gl/user_project_wrapper.v
./verilog/gl/user_proj_example.v
./verilog/dv/la_test/la_test_tb.v
./tapeout/outputs/verilog/rtl/caravel.v
./tapeout/outputs/verilog/gl/gpio_defaults_block_0403.v
./tapeout/outputs/verilog/gl/gpio_defaults_block_1803.v
./tapeout/outputs/verilog/gl/caravel.v
./tapeout/outputs/verilog/gl/user_id_programming.v


# Create lvs_config.json

Update the TOP_LAYOUT, LVS_SPICE_FILES, LVS_VERILOG_FILES, and LAYOUT_FILE for every design.

Update the other parameters as needed.

In [ ]:
%%writefile lvs_config.json
{
        "STD_CELL_LIBRARY": "sky130_fd_sc_hd",
        "INCLUDE_CONFIGS": [
                "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
        ],
        "TOP_SOURCE": "$TOP",
        "TOP_LAYOUT": "SJ_user_project_wrapper",
        "EXTRACT_FLATGLOB": [
                ""
        ],
        "EXTRACT_ABSTRACT": [
                ""
        ],
        "LVS_FLATTEN": [
                ""
        ],
        "LVS_NOFLATTEN": [
                ""
        ],
        "LVS_IGNORE": [
                ""
        ],
        "LVS_SPICE_FILES": [
                ""
        ],
        "LVS_VERILOG_FILES": [
                "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
                "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
        ],
        "LAYOUT_FILE": "$UPRJ_ROOT/work/gds/SJ_user_project_wrapper.gds.gz"
}

Writing lvs_config.json


# Run BE checks

In [ ]:
%%shell
cat /content/env
source /content/env
cd data
gdspath=$(ls work/gds/*project_wrapper.gds.gz)
gdsfile=${gdspath##*/}
layout=${gdsfile%%.gds.gz}
source=${layout//[A-Z0-9][A-Z0-9]_}
echo "Layout is $layout, source is $source"
cp $LVS_ROOT/tech/$PDK/cvc.power.$source work/cvc.power.$layout
cp ../lvs_config.json lvs_config.json
export INPUT_DIRECTORY=$PWD
sudo -u lvs udocker run -i -t -v $PRECHECK_ROOT:$PRECHECK_ROOT \
  -v $INPUT_DIRECTORY:$INPUT_DIRECTORY \
  -v $LVS_ROOT:$LVS_ROOT \
  -v $PDK_ROOT:$PDK_ROOT \
  -e INPUT_DIRECTORY=$INPUT_DIRECTORY \
  -e PDK_PATH=$PDK_ROOT/$PDK \
  -e PDK=$PDK \
  -e TOP=$TOP \
  -e PDK_ROOT=$PDK_ROOT \
  -e UPRJ_ROOT=$UPRJ_ROOT \
  -e PDKPATH=$PDKPATH \
  -e LVS_ROOT=$LVS_ROOT \
  -e WORK_ROOT=$INPUT_DIRECTORY/work \
  -u $(id -u lvs):$(id -g lvs) \
  efabless/mpw_precheck:latest bash -c "env; cd $INPUT_DIRECTORY ; $LVS_ROOT/run_be_checks lvs_config.json"
gunzip -f work/cvc.error.gz
head -1 work/cvc.oeb.report > work/cvc.oeb.sort.report
tail -n +2 work/cvc.oeb.report | sort -k1,1n >> work/cvc.oeb.sort.report

Streaming output truncated to the last 5000 lines.
  Class: sky130_fd_pr__nfet_01v8 instances:   8
  Class: sky130_fd_pr__pfet_01v8_hvt instances:  12
  Class: sky130_fd_pr__special_nfet_01v8 instances:   4
Circuit contains 16 nets.

Circuit 1 contains 24 devices, Circuit 2 contains 24 devices.
Circuit 1 contains 16 nets,    Circuit 2 contains 16 nets.


Contents of circuit 1:  Circuit: 'SJ_QD_sky130_fd_sc_hd__decap_3'
Circuit SJ_QD_sky130_fd_sc_hd__decap_3 contains 2 device instances.
  Class: sky130_fd_pr__nfet_01v8 instances:   1
  Class: sky130_fd_pr__pfet_01v8_hvt instances:   1
Circuit contains 2 nets.
Contents of circuit 2:  Circuit: 'SJ_QD_sky130_fd_sc_hd__decap_3'
Circuit SJ_QD_sky130_fd_sc_hd__decap_3 contains 2 device instances.
  Class: sky130_fd_pr__nfet_01v8 instances:   1
  Class: sky130_fd_pr__pfet_01v8_hvt instances:   1
Circuit contains 2 nets.

Circuit 1 contains 2 devices, Circuit 2 contains 2 devices.
Circuit 1 contains 2 nets,    Circuit 2 contains 2 nets.


Conte

# View verification logs

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')
files.view('/content/data/work/cvc.oeb.log')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

<IPython.core.display.Javascript object>

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

<IPython.core.display.Javascript object>

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')

<IPython.core.display.Javascript object>

# View OEB report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.oeb.sort.report')

<IPython.core.display.Javascript object>